In [2]:
import h5py
import numpy as np
import pandas as pd
import tables
import os
import sys
import time
import glob
import scipy.io as sio
import hdf5_getters
from pandas import read_hdf
from musixmatch import Musixmatch
# from musixmatch import Musixmatch

In [53]:
### Need to include track_id in pickle file #######
finalWorkingDF = pd.read_pickle("DFwithTrackIDs")
#finalWorkingDF.to_pickle("musicMatchDB")
#finalWorkingDF["words"] = np.nan

In [54]:
finalWorkingDF.columns.values

array(['artist_hotttnesss', 'danceability', 'duration', 'end_of_fade_in',
       'energy', 'key', 'loudness', 'mode', 'song_hotttnesss', 'tempo',
       'time_signature', 'year', 'Tim1', 'Tim2', 'Tim3', 'Tim4', 'Tim5',
       'Tim6', 'Tim7', 'Tim8', 'Tim9', 'Tim10', 'Tim11', 'Tim12',
       'Pitch1', 'Pitch2', 'Pitch3', 'Pitch4', 'Pitch5', 'Pitch6',
       'Pitch7', 'Pitch8', 'Pitch9', 'Pitch10', 'Pitch11', 'Pitch12',
       'words', 'track_id'], dtype=object)

In [30]:
match = dict()
musixmatchIDs = []
trackIDs = []
count = 1
with open('mxm_779k_matches.txt','r', encoding="utf8") as mxm_matches:
    line = mxm_matches.readline()
    while line:
        if line.startswith('#'):
            line = mxm_matches.readline()
            continue
        count += 1
        tokens = line.split("<SEP>")
        trackIDs.append(tokens[0])
        musixmatchIDs.append(tokens[3])
        line = mxm_matches.readline()
        
mxm_matches.close()

In [31]:
match["musixmatchIDs"] = musixmatchIDs
match["trackIDs"] = trackIDs
matchFrame = pd.DataFrame(match)

In [32]:
matchFrame.head(10000)

,musixmatchIDs,trackIDs
0,4418550,TRMMMKD128F425225D
1,8898149,TRMMMRX128F93187D9
2,9239868,TRMMMCH128F425532C
3,5346741,TRMMMXN128F42936A5
4,2511405,TRMMMBB12903CB7D21
5,793273,TRMMMHY12903CB53F1
6,7503609,TRMMMNS128F93548E1
7,7362052,TRMMMXJ12903CBF111
8,7833814,TRMMMBW128F4260CAE
9,2320200,TRMMMXI128F4285A3F


In [85]:
musixMatchforSubset = []

for row in finalWorkingDF["track_id"]:
    decodeR = row.decode("utf8")
    valIndex = np.where(decodeR==matchFrame["trackIDs"])[0]
    if valIndex.size<=0:
        valIndex = -1
        musixMatchforSubset.append("NOT HERE")
    else:
        valIndex = valIndex[0]
        musixMatchforSubset.append(matchFrame["musixmatchIDs"][valIndex])

musixMatchforSubset

['7005184',
 '3097978',
 '2985357',
 '7253531',
 'NOT HERE',
 '2253436',
 '1607604',
 'NOT HERE',
 '1509319',
 '8647996',
 '2573861',
 '4513723',
 '2663442',
 '665851',
 '1765518',
 '8412735',
 '1935814',
 '5050133',
 '2632746',
 'NOT HERE',
 'NOT HERE',
 '686207',
 '13302195',
 '5549136',
 '4583289',
 '1046140',
 '5316621',
 '5862496',
 '6768090',
 'NOT HERE',
 '9351871',
 '2620438',
 '8603415',
 '8025914',
 'NOT HERE',
 'NOT HERE',
 '4250905',
 '2623844',
 '1173124',
 'NOT HERE',
 '2910480',
 '6422381',
 'NOT HERE',
 '8508624',
 '12294540',
 '657458',
 'NOT HERE',
 '12458645',
 'NOT HERE',
 'NOT HERE',
 '3203905',
 '752458',
 '8957153',
 '4338007',
 '10104465',
 'NOT HERE',
 '8312752',
 '7735503',
 'NOT HERE',
 '4214072',
 '2963100',
 'NOT HERE',
 '5323016',
 '12857737',
 '710913',
 '7797067',
 '1750735',
 '4885749',
 '2541383',
 'NOT HERE',
 '1134355',
 '2872681',
 '6194025',
 'NOT HERE',
 '6995464',
 '9123709',
 '976236',
 '587145',
 '1631694',
 '5288588',
 '8764943',
 '924878',
 '

In [86]:
finalWorkingDF["musixIndex"] =  musixMatchforSubset
finalWorkingDF

,artist_hotttnesss,danceability,duration,end_of_fade_in,energy,key,loudness,mode,song_hotttnesss,tempo,...,Pitch6,Pitch7,Pitch8,Pitch9,Pitch10,Pitch11,Pitch12,words,track_id,musixIndex
0,0.386152,0.0,148.74077,0.192,0.0,0.0,-9.636,1.0,0.547953,124.059,...,2.485114,-68.224418,101.118626,10.091520,-28.738076,-120.949465,15.555821,NaN,b'TRARRZU128F4253CA2',7005184
1,0.434860,0.0,252.99546,0.514,0.0,1.0,-11.061,0.0,0.475638,80.084,...,-39.566763,-54.708540,100.295846,-18.160233,-15.933118,-29.334377,-18.621381,NaN,b'TRARRJL128F92DED0E',3097978
2,0.000000,0.0,78.02730,0.974,0.0,3.0,-24.140,1.0,NaN,54.874,...,-37.051717,-15.278711,90.671861,-12.548848,-15.388449,-24.970513,-1.573827,NaN,b'TRARRUZ128F9307C57',2985357
3,0.334520,0.0,163.63057,0.000,0.0,7.0,-5.795,1.0,NaN,77.150,...,-94.723603,-69.220156,124.548317,17.161881,-61.504485,-46.370004,-11.512016,NaN,b'TRARRWA128F42A0195',7253531
4,0.311616,0.0,199.99302,0.000,0.0,10.0,-16.477,1.0,NaN,120.382,...,29.126305,-50.517108,136.288091,-27.579095,11.283508,-67.039631,-5.180964,NaN,b'TRARRPG12903CD1DE9',NOT HERE
5,0.363676,0.0,279.35302,0.433,0.0,9.0,-12.474,1.0,0.445455,99.024,...,-39.805177,-6.645192,19.318075,-9.543714,8.518998,-27.863320,-53.757678,NaN,b'TRARRER128F9328521',2253436
6,0.553566,0.0,255.03302,0.449,0.0,9.0,-4.393,1.0,0.327737,175.673,...,5.659714,-24.409670,-11.275028,59.984377,-7.842671,-84.286842,31.783036,NaN,b'TRARRYC128F428CCDA',1607604
7,0.440135,0.0,259.31710,0.223,0.0,1.0,-5.050,1.0,NaN,87.999,...,35.644920,-11.693613,116.515463,-72.352900,-19.890188,-38.008860,-24.912274,NaN,b'TRARROY128F42281F7',NOT HERE
8,0.539245,0.0,216.84200,0.000,0.0,10.0,-4.264,1.0,0.788388,92.897,...,-1.378320,-49.908081,129.289499,18.306592,-37.477745,-118.501958,17.540179,NaN,b'TRARREF128F422FD96',1509319
9,0.358149,0.0,312.99873,0.000,0.0,4.0,-13.885,0.0,NaN,86.981,...,-43.720642,3.154264,-37.320277,-11.411132,8.590669,-16.934331,4.201740,NaN,b'TRARRVB128F92F47CA',8647996


In [91]:
dropRows = np.where("NOT HERE"==finalWorkingDF['musixIndex'])[0]
finalWorkingDF = finalWorkingDF.drop(dropRows)
finalWorkingDF.shape

(7506, 39)

In [94]:
finalWorkingDF = finalWorkingDF.drop('words', axis=1)
finalWorkingDF.to_pickle("DFwithTrackIDs")

In [95]:
aseoifn = pd.read_pickle("DFwithTrackIDs")
aseoifn

,artist_hotttnesss,danceability,duration,end_of_fade_in,energy,key,loudness,mode,song_hotttnesss,tempo,...,Pitch5,Pitch6,Pitch7,Pitch8,Pitch9,Pitch10,Pitch11,Pitch12,track_id,musixIndex
0,0.386152,0.0,148.74077,0.192,0.0,0.0,-9.636,1.0,0.547953,124.059,...,-12.774534,2.485114,-68.224418,101.118626,10.091520,-28.738076,-120.949465,15.555821,b'TRARRZU128F4253CA2',7005184
1,0.434860,0.0,252.99546,0.514,0.0,1.0,-11.061,0.0,0.475638,80.084,...,31.597252,-39.566763,-54.708540,100.295846,-18.160233,-15.933118,-29.334377,-18.621381,b'TRARRJL128F92DED0E',3097978
2,0.000000,0.0,78.02730,0.974,0.0,3.0,-24.140,1.0,NaN,54.874,...,-3.919090,-37.051717,-15.278711,90.671861,-12.548848,-15.388449,-24.970513,-1.573827,b'TRARRUZ128F9307C57',2985357
3,0.334520,0.0,163.63057,0.000,0.0,7.0,-5.795,1.0,NaN,77.150,...,-0.021905,-94.723603,-69.220156,124.548317,17.161881,-61.504485,-46.370004,-11.512016,b'TRARRWA128F42A0195',7253531
5,0.363676,0.0,279.35302,0.433,0.0,9.0,-12.474,1.0,0.445455,99.024,...,33.619536,-39.805177,-6.645192,19.318075,-9.543714,8.518998,-27.863320,-53.757678,b'TRARRER128F9328521',2253436
6,0.553566,0.0,255.03302,0.449,0.0,9.0,-4.393,1.0,0.327737,175.673,...,2.065868,5.659714,-24.409670,-11.275028,59.984377,-7.842671,-84.286842,31.783036,b'TRARRYC128F428CCDA',1607604
8,0.539245,0.0,216.84200,0.000,0.0,10.0,-4.264,1.0,0.788388,92.897,...,-26.360939,-1.378320,-49.908081,129.289499,18.306592,-37.477745,-118.501958,17.540179,b'TRARREF128F422FD96',1509319
9,0.358149,0.0,312.99873,0.000,0.0,4.0,-13.885,0.0,NaN,86.981,...,54.162901,-43.720642,3.154264,-37.320277,-11.411132,8.590669,-16.934331,4.201740,b'TRARRVB128F92F47CA',8647996
10,0.592439,0.0,218.90567,0.000,0.0,0.0,-4.707,0.0,0.681092,157.715,...,-40.821703,33.793109,-24.566356,28.760109,45.108962,-20.132572,-79.878481,-1.785747,b'TRARRQO128F427B5F5',2573861
11,0.389499,0.0,580.70159,5.294,0.0,0.0,-4.523,1.0,0.401487,146.331,...,-57.594836,-17.390071,-2.170541,-25.469523,32.338960,8.923370,5.358580,-19.895748,b'TRARRMK12903CDF793',4513723


In [101]:
newD = np.where(0==finalWorkingDF['year'])[0]
newD
#finalWorkingDF = finalWorkingDF.drop(newD)
finalWorkingDF.to_pickle("DFwithTrackIDs")

In [ ]:
api_key="4a008f1020fb22038ed6dc9328dd460e"
musixmatch = Musixmatch(api_key)
lyricsL = []
for index, row in matchFrame.iterrows():
    val = musixmatch.track_lyrics_get(row[0])
    lyrics = val['message']['body']
    if not lyrics:
        lyrics = "NOT HERE"
    else:
        lyrics = lyrics['lyrics']['lyrics_body']
        for letNum in range(len(lyrics)):
            if lyrics[letNum:letNum+3] == '...':
                lyrics = lyrics[:letNum]
    lyricsL.append(lyrics)

In [32]:
# finalWorkingDF["track_id"][0] == b'TRARRZU128F4253CA2'
# map(decode("utf-8"),finalWorkingDF["track_id"])
# finalWorkingDF["track_id"].decode("utf-8")

NameError: name 'decode' is not defined

In [39]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

database = "mxm_dataset2.db"
conn = create_connection(database)
cur = conn.cursor()

cur.execute("SELECT * FROM lyrics")
# count = 0
# for row in cur:
#     print(row)
#     count += 1
#     if count==600:
#         break

# print("break")
cur



In [38]:
count = 0
for row in cur:
    print(row[0].encode("utf-8"))
    count += 1
    if(count >5):
        break

b'TRAAAAV128F421A322'
b'TRAAAAV128F421A322'
b'TRAAAAV128F421A322'
b'TRAAAAV128F421A322'
b'TRAAAAV128F421A322'
b'TRAAAAV128F421A322'


In [41]:
corpus = []
currID = "bogus"
#vectorizer = CountVectorizer()
count = 0
newStr = ""
#for row
# if outside:
#  continue
# if same as before:
#  add correctly
# if different:
#  
inside = True
for row in cur:
    if not inside and currID == row[0]:
        continue
    if currID != row[0]:
        if row[0].encode("utf-8") in finalWorkingDF["track_id"]:
            inside = True
            print("found new")
        else:
            inside = False
            continue
        if count != 0:
            corpus.append(newStr)
        newStr = row[3]*(row[2]+" ")
        currID = row[0]
        count += 1
        print(str(count))
    else:
        newStr += row[3]*(row[2]+" ")
        
    if count == 10:
        break

        
#X = vectorizer.fit_transform(corpus)
#print(vectorizer.get_feature_names())
#print(X.toarray())

# vectorizer = CountVectorizer(analyzer = "word",   \
#                              tokenizer = None,    \
#                              preprocessor = None, \
#                              stop_words = None,   \
#                              max_features = 5000) 

# # fit_transform() does two functions: First, it fits the model
# # and learns the vocabulary; second, it transforms our training data
# # into feature vectors. The input to fit_transform should be a list of 
# # strings.
# train_data_features = vectorizer.fit_transform(corpus)

# # Numpy arrays are easy to work with, so convert the result to an 
# # array
# train_data_features = train_data_features.toarray()

# # see final the clean data
# print (train_data_features.shape)
    
# vocab = vectorizer.get_feature_names()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents
text = ["The quick brown fox jumped over the lazy dog."]
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

{'the': 7, 'quick': 6, 'brown': 0, 'fox': 2, 'jumped': 3, 'over': 5, 'lazy': 4, 'dog': 1}
(1, 8)
<class 'scipy.sparse.csr.csr_matrix'>
[[1 1 1 1 1 1 1 2]]
{'the': 7, 'quick': 6, 'brown': 0, 'fox': 2, 'jumped': 3, 'over': 5, 'lazy': 4, 'dog': 1}


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer()
vect

# use TreeankWordTokenizer
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
vect.set_params(tokenizer=tokenizer.tokenize)

# remove English stop words
vect.set_params(stop_words='english')

# include 1-grams and 2-grams
vect.set_params(ngram_range=(1, 2))

# ignore terms that appear in more than 50% of the documents
vect.set_params(max_df=0.5)

# only keep terms that appear in at least 2 documents
vect.set_params(min_df=2)

In [ ]:
# data.merge(genres,on='track_id')
usedSongLyrics = pd.DataFrame(columns=['TrackID','MusicMatchID','Word','Count','Training/Testing'])

count = 0
for row in cur:
    if row[0] in trackIDs:
        usedSongLyrics.loc[count] = row
        count += 1


